## Plan and Execute

Agent that generates analysis reports using Plan and Execute Agent Workflow

In [ ]:
!pip install -q langchain openai tiktoken
!pip install -qU langchain-openai
!pip install -U langchain-community
!pip install newspaper3k
!pip install --upgrade lxml_html_clean

In [ ]:
import os
import getpass
import openai

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [ ]:
os.environ["ACTIVELOOP_TOKEN"] = getpass.getpass("Activeloop Token:")

Activeloop Token:··········


In [ ]:
import requests
from newspaper import Article
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/news/western-drivers-remain-sceptical-in-vehicle-ai/",
    "https://www.artificialintelligence-news.com/news/how-to-use-ai-driven-speech-analytics-contact-centres/"
    "https://www.artificialintelligence-news.com/news/sas-aims-to-make-ai-accessible-regardless-of-skill-set-with-packaged-ai-models/",
    "https://www.artificialintelligence-news.com/news/meta-open-source-speech-ai-models-support-over-1100-languages/",
    "https://www.artificialintelligence-news.com/news/ai21-labs-raises-64m-to-help-it-compete-against-openai/",
    "https://www.artificialintelligence-news.com/news/ai-hallucinations-gone-wrong-as-alaska-uses-fake-stats-in-policy/",
    "https://www.artificialintelligence-news.com/news/project-jarvis-leak-reveals-google-vision-for-the-next-gen-gemini/",
    "https://www.artificialintelligence-news.com/news/anthropic-urges-ai-regulation-avoid-catastrophes/",
    "https://www.artificialintelligence-news.com/news/github-expands-ai-suite-copilot-goes-multi-model-spark-debuts-for-web-dev/",
    "https://www.artificialintelligence-news.com/news/industry-leaders-back-open-source-ai-definition/",
    "https://www.artificialintelligence-news.com/news/mit-breakthrough-could-transform-robot-training/",
    "https://www.artificialintelligence-news.com/news/president-biden-issues-first-national-security-memorandum-ai/",
    "https://www.artificialintelligence-news.com/news/ai-is-helping-brands-avoid-controversial-influencer-partnerships/",
    "https://www.artificialintelligence-news.com/news/intern-allegedly-sabotages-bytedance-ai-project-leading-to-dismissal/"
]


session = requests.Session()
pages_content = []

for url in article_urls:
  try:
    time.sleep(2)
    response = session.get(url, headers=headers, timeout = 10)

    if response.status_code == 200:
      article = Article(url)
      article.download()
      article.parse()
      pages_content.append({"url": url, "text": article.text})
    else:
        print(f"Failed to fetch article at {url}")
  except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

In [ ]:
pages_content

[{'url': 'https://www.artificialintelligence-news.com/news/western-drivers-remain-sceptical-in-vehicle-ai/',
  'text': 'Ryan Daws is a senior editor at TechForge Media with over a decade of experience in crafting compelling narratives and making complex topics accessible. His articles and interviews with industry leaders have earned him recognition as a key influencer by organisations like Onalytica. Under his leadership, publications have been praised by analyst firms such as Forrester for their excellence and performance. Connect with him on X (@gadget_ry) or Mastodon (@gadgetry@techhub.social)\n\nA global study has unveiled a stark contrast in attitudes towards embracing in-vehicle AI between Eastern and Western markets, with European drivers particularly reluctant.\n\nThe research – conducted by MHP – surveyed 4,700 car drivers across China, the US, Germany, the UK, Italy, Sweden, and Poland, revealing significant geographical disparities in AI acceptance and understanding.\n\nAcco

In [ ]:
!pip install deeplake==3.9.26

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")

# create Deep Lake dataset
my_activeloop_org_id = "jacobasir"
my_activeloop_dataset_name = "agents_for_creating_analysis_reports"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

/usr/local/lib/python3.10/dist-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.0.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Your Deep Lake dataset has been successfully created!


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap =100)
all_texts = []
for i in pages_content:
  chunks = text_splitter.split_text(i["text"])
  for chunk in chunks:
    all_texts.append(chunk)

In [ ]:
all_texts

['Ryan Daws is a senior editor at TechForge Media with over a decade of experience in crafting',
 'Daws is a senior editor at TechForge Media with over a decade of experience in crafting compelling',
 'senior editor at TechForge Media with over a decade of experience in crafting compelling narratives',
 'editor at TechForge Media with over a decade of experience in crafting compelling narratives and',
 'at TechForge Media with over a decade of experience in crafting compelling narratives and making',
 'Media with over a decade of experience in crafting compelling narratives and making complex topics',
 'over a decade of experience in crafting compelling narratives and making complex topics accessible.',
 'a decade of experience in crafting compelling narratives and making complex topics accessible. His',
 'of experience in crafting compelling narratives and making complex topics accessible. His articles',
 'experience in crafting compelling narratives and making complex topics accessib

In [ ]:
#adding chunks to deeplake
db.add_texts(all_texts)

Creating 4132 embeddings in 9 batches of size 500:: 100%|██████████| 9/9 [02:27<00:00, 16.37s/it]

Dataset(path='hub://jacobasir/agents_for_creating_analysis_reports', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
   text       text      (4132, 1)      str     None   
 metadata     json      (4132, 1)      str     None   
 embedding  embedding  (4132, 1536)  float32   None   
    id        text      (4132, 1)      str     None   


['6a9af726-aaff-11ef-90ba-0242ac1c000c',
 '6a9af94c-aaff-11ef-90ba-0242ac1c000c',
 '6a9afa3c-aaff-11ef-90ba-0242ac1c000c',
 '6a9afaf0-aaff-11ef-90ba-0242ac1c000c',
 '6a9afb9a-aaff-11ef-90ba-0242ac1c000c',
 '6a9afc30-aaff-11ef-90ba-0242ac1c000c',
 '6a9afcc6-aaff-11ef-90ba-0242ac1c000c',
 '6a9afd66-aaff-11ef-90ba-0242ac1c000c',
 '6a9afdfc-aaff-11ef-90ba-0242ac1c000c',
 '6a9afe9c-aaff-11ef-90ba-0242ac1c000c',
 '6a9aff32-aaff-11ef-90ba-0242ac1c000c',
 '6a9affd2-aaff-11ef-90ba-0242ac1c000c',
 '6a9b0072-aaff-11ef-90ba-0242ac1c000c',
 '6a9b0112-aaff-11ef-90ba-0242ac1c000c',
 '6a9b01b2-aaff-11ef-90ba-0242ac1c000c',
 '6a9b0252-aaff-11ef-90ba-0242ac1c000c',
 '6a9b02e8-aaff-11ef-90ba-0242ac1c000c',
 '6a9b0388-aaff-11ef-90ba-0242ac1c000c',
 '6a9b04d2-aaff-11ef-90ba-0242ac1c000c',
 '6a9b057c-aaff-11ef-90ba-0242ac1c000c',
 '6a9b0630-aaff-11ef-90ba-0242ac1c000c',
 '6a9b06da-aaff-11ef-90ba-0242ac1c000c',
 '6a9b077a-aaff-11ef-90ba-0242ac1c000c',
 '6a9b082e-aaff-11ef-90ba-0242ac1c000c',
 '6a9b08d8-aaff-

### Creating Plan and Execute Agent

In [ ]:
retriever = db.as_retriever()
retriever.search_kwargs["k"] = 3

In [ ]:
CUSTOM_TOOL_DOCS_SEPARATOR ="\n---------------\n"

# this function accepts query and uses retriever to locate docs relevant to query
def retrieve_n_docs_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)
    texts = [doc.page_content for doc in docs]
    texts_merged = "---------------\n" + CUSTOM_TOOL_DOCS_SEPARATOR.join(texts) + "\n---------------" #text of the retrieved docs is then concatenated  CUSTOM_TOOL_DOCS_SEPARATOR then transforms the docs imto single text
    return texts_merged

In [ ]:
from langchain.agents import Tool

tools = [
    Tool(
        name = "Search Private Docs",
        func = retrieve_n_docs_tool,
        description = "useful for when you need to answer questions about current events about Artificial Intelligence"
    )
]

In [ ]:
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

model = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose = True)
agent = PlanAndExecute(planner = planner, executor = executor, verbose = True)

<ipython-input-20-7c917b589be4>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)


In [ ]:
response = agent.run("Write an overview of GitHub expands AI suite: Copilot")

<ipython-input-21-844ada8d0cf4>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("Write an overview of GitHub expands AI suite: Copilot")




> Entering new PlanAndExecute chain...
steps=[Step(value='Research GitHub Copilot and its features.'), Step(value='Summarize the key points of GitHub Copilot, including its purpose, capabilities, and benefits.'), Step(value='Include information on how GitHub Copilot utilizes AI to assist developers.'), Step(value='Mention any potential concerns or limitations of GitHub Copilot.'), Step(value='Provide examples or use cases of how GitHub Copilot can be used in software development.'), Step(value='Conclude with a brief opinion or recommendation on GitHub Copilot.'), Step(value="Review and finalize the overview of GitHub Copilot.\nGiven the above steps taken, please respond to the user's original question. \n")]

> Entering new AgentExecutor chain...
Thought: I can search private documents to provide information about GitHub Copilot and its features.
Action:
```
{
  "action": "Search Private Docs",
  "action_input": "GitHub Copilot features"
}
```

<ipython-input-17-41ab3e49d740>:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)



Observation: ---------------
GitHub Copilot with Anthropic’s Claude 3.5 Sonnet, Google’s Gemini 1.5 Pro, and OpenAI’s o1-preview
---------------
Copilot’s initial release, this three-pronged update represents GitHub’s most ambitious AI toolkit
---------------
Since Copilot’s initial release, this three-pronged update represents GitHub’s most ambitious AI
---------------
Thought:Thought: I have found some information about GitHub Copilot and its features from the private documents. I can now summarize the key points for the user.
Action:
```
{
  "action": "Final Answer",
  "action_input": "GitHub Copilot is an AI tool developed by GitHub that includes features like Anthropic’s Claude 3.5 Sonnet, Google’s Gemini 1.5 Pro, and OpenAI’s o1-preview. It is considered GitHub's most ambitious AI toolkit and has seen significant updates since its initial release."
}
```

> Finished chain.
*****

Step: Research GitHub Copilot and its features.

Response: GitHub Copilot is an AI tool developed by

In [ ]:
print(response)

GitHub Copilot is an AI tool developed by GitHub that collaborates with Anthropic’s Claude 3.5 Sonnet, Google’s Gemini 1.5 Pro, and OpenAI’s o1-preview. It utilizes Codex, an early version of OpenAI GPT-3, specifically fine-tuned for coding tasks. GitHub Copilot aims to assist developers by providing intelligent code suggestions and assistance, potentially enhancing productivity. However, concerns include issues related to code quality, privacy implications, and the risk of overreliance on the tool. When used judiciously, GitHub Copilot can streamline coding workflows and accelerate software development processes.
